In [80]:
import pandas as pd
from unidecode import unidecode

## Peso y volumen de los libros

In [81]:
libros = pd.read_csv('../data/raw/PesoVolumenLibros.csv', encoding='latin1')
display(libros)

,libro_titulo,libro_isbn,ancho_cm,alto_cm,grosor_cm,paginas,vol_cm3_1libro,vol_m3_500libros,peso_g_1libro,peso_t_500libros
0,Niebla de sueño,978-84-1353-114-4,13,19,1.2,162,296.4,0.14820,250,0.125
1,La isla de los canarios. Volumen 6,978-84-1353-115-1,21,23,0.8,92,386.4,0.19320,400,0.200
2,El mundo del libro en Canarias,84-8103-396-0,15,21,3.1,532,976.5,0.48825,780,0.390
3,Historia del Cabildo insular de Gran Canaria,84-8103-067-8,17,24,3.9,748,1591.2,0.79560,1240,0.620
4,Flora de Gran Canaria 1,978-84-8103-741,27,34,3.0,238,2754.0,1.37700,2280,1.140


# Etapa 1: identificación de los diferentes trayectos de este servicio de transporte

**Ruta**. Camino (a ser) tomado para llegar de un punto de partida a un punto de destino. (AENOR2013)

**Trayecto**. Para un servicio de transporte de mercancías o de pasajeros, la sección de la ruta tomada o que se ha de tomar, dentro de la cual el vehículo transporta las mercancías o el pasajero (AENOR2013).

In [82]:
salida_carga_distance = pd.read_csv('../data/final/city_port_distance.csv')
salida_carga_distance.drop(columns=['Unnamed: 0', 'id', 'lat', 'lon'], inplace=True)
# Si el puerto más cercano es el de Las Palmas significa que la ciudad está en 
# Gran Canaria, por lo tanto se irá directamente al punto de despacho sin
# necesidad de ir al puerto (este trayecto será 0).
salida_carga_distance.loc[salida_carga_distance['port_name']=='LAS PALMAS', 'shortest_distance_to_port_km'] = 0 

carga_descarga_distance = pd.read_csv('../data/final/port_portlp_distance.csv')
carga_descarga_distance.drop(columns='Unnamed: 0', inplace=True)

descarga_llegada_distance = pd.read_csv('../data/final/puertodescarga_llegada_distance.csv')
descarga_llegada_distance.drop(columns='Unnamed: 0', inplace=True)

In [83]:
# Añadir datos del peso de los libros a transportar para cada trayecto
salida_carga = salida_carga_distance.copy()

carga_descarga = carga_descarga_distance.copy()
# carga_descarga = pd.merge(carga_descarga_distance, libros[['libro_titulo', 'peso_t_500libros']], how='cross')

descarga_llegada = descarga_llegada_distance.copy()

In [84]:
display('TrayectoSC por carretera', salida_carga.head(5))
display('TrayectoCD por mar', carga_descarga.head(5))
display('TrayectoDL por carretera', descarga_llegada)

'TrayectoSC por carretera'

,name,population,latlon,port_name,port_latlon,shortest_distance_to_port_km
0,FUENGIROLA,71482,"36.53998,-4.62473",ALGECIRAS,"36.13672,-5.434271",254.677
1,CALVIA,51774,"39.5657,2.50621",VALENCIA,"39.44231,-0.316466",284.566
2,ARONA,78614,"28.09962,-16.68102",LA GOMERA,"28.0655,-17.1175",56.399
3,PONTEVEDRA,81576,"42.431,-8.64435",BILBAO,"43.3558,-3.066499",631.209
4,GIRONA,96188,"41.98311,2.82493",BARCELONA,"41.35238,2.158441",110.221


'TrayectoCD por mar'

,port_name,distanceLP_nm,distanceLP_km
0,A CORUNA,1001,1853.852
1,ALGECIRAS,702,1300.104
2,ALICANTE,997,1846.444
3,ALMERIA,848,1570.496
4,AVILES,1098,2033.496


'TrayectoDL por carretera'

,port_name,port_latlon,punto_llegada_latlon,descarga_llegada_distancia_km
0,LAS PALMAS,"28.141,-15.41635","28.086528,-15.434806",13.961


# Etapa 2: cálculo del consumo de energía y de emisiones de GEI de cada trayecto

## Subetapa 2.1: establecimiento del sitema de operación de un vehículo (VOS) relativo a este trayecto

### 1. Trayecto salida-carga

- Trayecto 1 - VOS de ciudad a puerto por carretera
- A 12-24 t lorry transports a container (1 TEU) with average goods. El contenedor pesa 1.95 toneladas, y lleva una carga de 10.5 toneladas, por lo tanto, su peso total es 12.45 toneladas (Table 9, (Schmied2012))
- El contenedor está cargado al 60%. **Factor de carga**. Ratio obtenido por la relación entre la carga real y la carga máxima autorizada de un medio de transporte. (AENOR2013)
- Specific energy consumption (Table 12, (Schimied2012)) = 0.24 liter/TEU-km. Ya considera Additional empty trips +10% (Schmied2012).
- Asumimos que el camión viaja mayoritariamente por autopistas/autovías, por terreno plano. No aplicamos factor de corrección por el uso de carreteras urbanas.
- El motor es de diesel (no biofuel).
- Calculamos sólo ida.

### 2. Trayecto carga-descarga

- Trayecto 2 - VOS de puerto a Puerto de Las Palmas
- Un buque RoRo 

### 3. Trayecto descarga-llegada

- Trayecto 3 - VOS de Puerto de Las Palmas a punto de despacho
- Un camión < 7.5 t GVW transporta average goods tiene un maximum payload de 3.5 t y como va al 60% de su capacidad total, la carga que transporta pesa 2.1 t. Additional empty trips = 20% (Schmied2012, Tabla 10)
- Asumimos que el terreno por el que circula es relativamente plano.
- Transporta average goods
- En base a lo anterior, specific energy consumption litre/tkm = 0.077
- Este trayecto se realiza en Gran Canaria (puerto-llegada) y aplicamos un factor de corrección  (Schmied2012, Tabla 11). El factor de corrección es 0.9.
- El camión es de diesel.

## Subetapa 2.2: cuantificación del consumo total de combustible para el VOS

- La distancia a cubrir es entre las ciudades de origen en la península y en el resto de islas Canarias (no GC) está en la tabla *salida_carga_distance*.
- Para el cálculo del consumo de combustible utilizaremos el *consumo energético específico* o *specific energy consumption*. Table 12 de (Schimied2012)

### 1. Trayecto salida-carga

In [85]:
# Definir funcion para el cálculo del consumo de diesel en base a la distancia.
def salida_carga_diesel(distancia_trayecto):
    specific_energy_consumption_lTEUkm = 0.24
    diesel_l = distancia_trayecto * specific_energy_consumption_lTEUkm
    return diesel_l

In [86]:
salida_carga['consumo_diesel_TEU_l'] = salida_carga.apply(lambda row: salida_carga_diesel(row['shortest_distance_to_port_km']), axis=1)

### 2. Trayecto carga-descarga

### 3. Trayecto descarga-llegada

In [87]:
# Definir funcion para el cálculo del consumo de diesel en base a la distancia.
def descarga_llegada_diesel(distancia_trayecto):
    specific_energy_consumption_ltkm = 0.077
    actual_cargo_weight_t = 2.1
    factor_correccion_carreteras_urbanas = 0.9
    diesel_l = (distancia_trayecto * actual_cargo_weight_t * specific_energy_consumption_ltkm) * factor_correccion_carreteras_urbanas
    return diesel_l

In [88]:
descarga_llegada['consumo_diesel_total_l'] = descarga_llegada.apply(lambda row: descarga_llegada_diesel(row['descarga_llegada_distancia_km']), axis=1)

## Subetapa 2.3: cálculo del consumo total de energía y de emisiones de GEI para este VOS

En base al consumo de combustible, se calcula:
1. Consumo de energía de total (MJ). Para esto se multiplica por los factores energéticos (WTW y TTW).
2. Emisiones de carbono (kgCO2/e). Para esto se multiplica por los factores de emisión (WTW y TTW).

### 1. Trayecto salida-carga

#### Energía total

In [89]:
# Datos (AENOR2013)
def carretera_consumo_energetico_TTW(consumo_diesel):
    factor_energetico_TTW_et_MJl = 35.9
    carretera_TTW_MJ = consumo_diesel * factor_energetico_TTW_et_MJl
    return carretera_TTW_MJ

In [90]:
def carretera_consumo_energetico_WTW(consumo_diesel):
    factor_energetico_WTW_ew_MJl = 42.7
    carretera_WTW_MJ = consumo_diesel * factor_energetico_WTW_ew_MJl
    return carretera_WTW_MJ

In [91]:
def carretera_consumo_energetico_WTT(consumo_diesel):
    wtw = carretera_consumo_energetico_WTW(consumo_diesel)
    ttw = carretera_consumo_energetico_TTW(consumo_diesel)
    wtt = wtw - ttw
    return wtt

#### Emisiones carbono

In [92]:
# Datos (AENOR2013)
def carretera_emisiones_carbono_TTW(consumo_diesel):
    factor_emision_TTW_gt_kgCO2el = 2.67
    carretera_TTW_kgCO2e = consumo_diesel * factor_emision_TTW_gt_kgCO2el
    return carretera_TTW_kgCO2e

In [93]:
def carretera_emisiones_carbono_WTW(consumo_diesel):
    factor_emision_WTW_gw_kgCO2el = 3.24
    carretera_WTW_kgCO2e = consumo_diesel * factor_emision_WTW_gw_kgCO2el
    return carretera_WTW_kgCO2e

In [94]:
def carretera_emisiones_carbono_WTT(consumo_diesel):
    wtw = carretera_emisiones_carbono_WTW(consumo_diesel)
    ttw = carretera_emisiones_carbono_TTW(consumo_diesel)
    wtt = wtw - ttw
    return wtt

In [95]:
salida_carga['emisionesC_TTW_TEU_kgCO2e'] = salida_carga.apply(lambda row: carretera_emisiones_carbono_TTW(row['consumo_diesel_TEU_l']), axis=1)

salida_carga['emisionesC_WTT_TEU_kgCO2e'] = salida_carga.apply(lambda row: carretera_emisiones_carbono_WTT(row['consumo_diesel_TEU_l']), axis=1)

salida_carga['emisionesC_WTW_TEU_kgCO2e'] = salida_carga.apply(lambda row: carretera_emisiones_carbono_WTW(row['consumo_diesel_TEU_l']), axis=1)

### 2. Trayecto carga-descarga

### 3. Trayecto descarga-llegada

#### Emisiones carbono

Aprovechamos las funciones definidas para el Tramo salida_carga.

In [96]:
descarga_llegada['emisionesC_TTW_total_kgCO2e'] = descarga_llegada.apply(lambda row: carretera_emisiones_carbono_TTW(row['consumo_diesel_total_l']), axis=1)

descarga_llegada['emisionesC_WTT_total_kgCO2e'] = descarga_llegada.apply(lambda row: carretera_emisiones_carbono_WTT(row['consumo_diesel_total_l']), axis=1)

descarga_llegada['emisionesC_WTW_total_kgCO2e'] = descarga_llegada.apply(lambda row: carretera_emisiones_carbono_WTW(row['consumo_diesel_total_l']), axis=1)

## Subetapa 2.4: asignación de este trayecto a una porción de cada uno de los cuatro resultados de la Subetapa 2.3

- Ahora se debe asignar una parte de la energía y de las emisiones calculadas,
correspondiente a su participación relativa en la actividad de transporte ralizada
dentro del VOS.
- Ver (AENOR2013) pg. 18.
    - Primero calculamos **S(trayecto)**, el factor utilizado para calcularla proporción
    de la energía y de las emisiones del VOS que se asignan a un servicio de transporte
    para el trayecto y para el VOS asociado.
    - S(trayecto) = T(trayecto) / T(VOS)
        - T(trayecto): actividade de transporte del servicio de transport para el trayecto
        - T(VOS): actividad de transporte del VOS relativo al trayecto
- Datos cálculo: 
    - Peso total del contenedor es 12.45 toneladas (Table 9, (Schmied2012)).
    - Peso de la mercancía transportada 0.125 toneladas. Share of diesel consumption per tonne cargo.

### 1. Trayecto salida-carga

#### Factor proporción libros

In [97]:
# Añadir peso de los libros
salida_carga = pd.merge(salida_carga, libros[['libro_titulo', 'peso_t_500libros']], how='cross')

In [98]:
def carretera_factor_proporcion_carga(peso_total, peso_mercancia):
    factor = peso_mercancia / peso_total
    return factor

#### Energía total

In [99]:
def carretera_consumo_energetico_proporcion_TTW(peso_total, peso_mercancia, consumo_diesel):
    factor = carretera_factor_proporcion_carga(peso_total, peso_mercancia)
    total = carretera_consumo_energetico_TTW(consumo_diesel)
    proporcion = factor * total
    return proporcion

In [100]:
def carretera_consumo_energetico_proporcion_WTW(peso_total, peso_mercancia, consumo_diesel):
    factor = carretera_factor_proporcion_carga(peso_total, peso_mercancia)
    total = carretera_consumo_energetico_WTW(consumo_diesel)
    proporcion = factor * total
    return proporcion

In [101]:
def carretera_consumo_energetico_proporcion_WTT(peso_total, peso_mercancia, consumo_diesel):
    factor = carretera_factor_proporcion_carga(peso_total, peso_mercancia)
    total = carretera_consumo_energetico_WTT(consumo_diesel)
    proporcion = factor * total
    return proporcion

#### Emisiones carbono

In [102]:
def carretera_emisiones_carbono_proporcion_TTW(peso_total, peso_mercancia, consumo_diesel):
    factor = carretera_factor_proporcion_carga(peso_total, peso_mercancia)
    total = carretera_emisiones_carbono_TTW(consumo_diesel)
    proporcion = factor * total
    return proporcion

In [103]:
def carretera_emisiones_carbono_proporcion_WTW(peso_total, peso_mercancia, consumo_diesel):
    factor = carretera_factor_proporcion_carga(peso_total, peso_mercancia)
    total = carretera_emisiones_carbono_WTW(consumo_diesel)
    proporcion = factor * total
    return proporcion

In [104]:
def carretera_emisiones_carbono_proporcion_WTT(peso_total, peso_mercancia, consumo_diesel):
    factor = carretera_factor_proporcion_carga(peso_total, peso_mercancia)
    total = carretera_emisiones_carbono_WTT(consumo_diesel)
    proporcion = factor * total
    return proporcion

In [105]:
# El contendor pesa en total 12.45 tonnes
salida_carga['emisionesC_TTW_libros_kgCO2e'] = salida_carga.apply(lambda row: carretera_emisiones_carbono_proporcion_TTW(peso_total=12.45, peso_mercancia=row['peso_t_500libros'], consumo_diesel=row['consumo_diesel_TEU_l']), axis=1)

salida_carga['emisionesC_WTT_libros_kgCO2e'] = salida_carga.apply(lambda row: carretera_emisiones_carbono_proporcion_WTT(peso_total=12.45, peso_mercancia=row['peso_t_500libros'], consumo_diesel=row['consumo_diesel_TEU_l']), axis=1)

salida_carga['emisionesC_WTW_libros_kgCO2e'] = salida_carga.apply(lambda row: carretera_emisiones_carbono_proporcion_WTW(peso_total=12.45, peso_mercancia=row['peso_t_500libros'], consumo_diesel=row['consumo_diesel_TEU_l']), axis=1)

### 2. Trayecto carga-descarga

### 3. Trayecto descarga-llegada

In [106]:
descarga_llegada = pd.merge(descarga_llegada, libros[['libro_titulo', 'peso_t_500libros']], how='cross')

Aprovechamos las funciones definidas para el Tramo salida_carga.

In [108]:
# Asumimos que el camión lleva una carga de 2.1 toneladas.
descarga_llegada['emisionesC_TTW_libros_kgCO2e'] = descarga_llegada.apply(lambda row: carretera_emisiones_carbono_proporcion_TTW(peso_total=2.1, peso_mercancia=row['peso_t_500libros'], consumo_diesel=row['consumo_diesel_total_l']), axis=1)

descarga_llegada['emisionesC_WTT_libros_kgCO2e'] = descarga_llegada.apply(lambda row: carretera_emisiones_carbono_proporcion_WTT(peso_total=2.1, peso_mercancia=row['peso_t_500libros'], consumo_diesel=row['consumo_diesel_total_l']), axis=1)

descarga_llegada['emisionesC_WTW_libros_kgCO2e'] = descarga_llegada.apply(lambda row: carretera_emisiones_carbono_proporcion_WTW(peso_total=2.1, peso_mercancia=row['peso_t_500libros'], consumo_diesel=row['consumo_diesel_total_l']), axis=1)